In [27]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta, date, time

my_free_api_key = '9a2c24d00b614ff1d74a08f03054ef53'

In [24]:
# get timestamp in correct format for tomorrow 8 AM until 8 PM
def get_tomorrow():
    date_tomorrow = date.today() + timedelta(days=1)
    time_start = time(hour=0, minute=0, second=0)
    time_middle = time(hour=12, minute=0, second=0)
    time_end = time(hour=23, minute=59, second=59)

    datetime_start = datetime.combine(date_tomorrow, time_start).strftime('%Y-%m-%dT%H:%M')
    datetime_middle = datetime.combine(date_tomorrow, time_middle).strftime('%Y-%m-%dT%H:%M')
    datetime_end = datetime.combine(date_tomorrow, time_end).strftime('%Y-%m-%dT%H:%M')
    return datetime_start,datetime_middle,datetime_end

In [34]:
def weather_forcast(list_of_cities):
      
    cities_list  = []
    weather_list = []
    
    for city in list_of_cities :
        city_r = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={my_free_api_key}&units=metric")
        city_j = city_r.json()
        
        cities_list.append(city_j['city'])
        

        for detail in city_j['list']:
            weather_details_dic = {}
            weather_details_dic["city_id"] = city_j['city']['id']
            weather_details_dic["city_name"] = city_j['city']['name']
            weather_details_dic["temp"] = detail['main']['temp']
            weather_details_dic["temp_min"] = detail['main']['temp_min']
            weather_details_dic["temp_max"] = detail['main']['temp_max']
            weather_details_dic["feels_like"] = detail['main']['feels_like']
            weather_details_dic["pressure"] = detail['main']['pressure']
            weather_details_dic["humidity"] = detail['main']['humidity']
            weather_details_dic["weather"] = detail['weather'][0]['main']
            weather_details_dic["weather_description"] = detail['weather'][0]['description']
            weather_details_dic["clouds"] = detail['clouds']['all']
            weather_details_dic["wind_speed"] = detail['wind']['speed']
            weather_details_dic["date_time"] = detail['dt_txt']
            weather_list.append(weather_details_dic)
    
    
    df_weather = pd.DataFrame(weather_list)
    dt_start,dt_middle,dt_end = get_tomorrow()
    df_weather['date_time'] = pd.to_datetime(df_weather['date_time'])
    df_weather = df_weather.loc[lambda df_ : (df_['date_time'] >= dt_start) &  (df_['date_time'] <= dt_end)]
    
    df_cities = pd.json_normalize(cities_list)
    
    
    return df_weather,df_cities

In [35]:
df1, df2 = weather_forcast(['LONDON'])

In [36]:
df1

,city_id,city_name,temp,temp_min,temp_max,feels_like,pressure,humidity,weather,weather_description,clouds,wind_speed,date_time
4,2643743,London,20.55,20.55,20.55,20.67,1014,77,Clouds,scattered clouds,28,2.78,2022-08-03 00:00:00
5,2643743,London,19.05,19.05,19.05,19.20,1013,84,Clouds,scattered clouds,31,2.78,2022-08-03 03:00:00
6,2643743,London,19.30,19.30,19.30,19.48,1014,84,Clouds,few clouds,22,4.02,2022-08-03 06:00:00
7,2643743,London,22.66,22.66,22.66,22.65,1014,64,Clouds,broken clouds,70,4.73,2022-08-03 09:00:00
8,2643743,London,25.76,25.76,25.76,25.64,1014,48,Clouds,broken clouds,81,3.93,2022-08-03 12:00:00
9,2643743,London,27.26,27.26,27.26,27.11,1013,41,Clouds,overcast clouds,99,3.39,2022-08-03 15:00:00
10,2643743,London,27.67,27.67,27.67,27.38,1012,40,Clouds,broken clouds,69,4.69,2022-08-03 18:00:00
11,2643743,London,22.81,22.81,22.81,22.40,1013,48,Clear,clear sky,10,3.33,2022-08-03 21:00:00


In [2]:
import requests

url = "https://aerodatabox.p.rapidapi.com/airports/search/term"

querystring = {"q":"london","limit":"10"}

headers = {
	"X-RapidAPI-Key": "5271551a49mshc194f542b6dc600p113722jsn431b32cb69ab",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"items":[{"icao":"EGSS","iata":"STN","name":"London, London Stansted","shortName":"Stansted","municipalityName":"London","location":{"lat":51.885,"lon":0.234999},"countryCode":"GB"},{"icao":"EGLL","iata":"LHR","name":"London, London Heathrow","shortName":"Heathrow","municipalityName":"London","location":{"lat":51.4706,"lon":-0.461941},"countryCode":"GB"},{"icao":"EGLC","iata":"LCY","name":"London, London City","shortName":"City","municipalityName":"London","location":{"lat":51.5053,"lon":0.055277},"countryCode":"GB"},{"icao":"EGKK","iata":"LGW","name":"London, London Gatwick","shortName":"Gatwick","municipalityName":"London","location":{"lat":51.1481,"lon":-0.190277},"countryCode":"GB"},{"icao":"EGGW","iata":"LTN","name":"London, London Luton","shortName":"Luton","municipalityName":"London","location":{"lat":51.8747,"lon":-0.368333},"countryCode":"GB"},{"icao":"KLOZ","iata":"LOZ","name":"London, London-Corbin /Magee Field","shortName":"-Corbin /Magee Field","municipalityName":"London"

In [14]:
airports_j = response.json()
# airports_j['items']

right_airports = [airport_j for airport_j in airports_j['items'] if airport_j['countryCode']=='GB'] 
right_airports


[{'icao': 'EGSS',
  'iata': 'STN',
  'name': 'London, London Stansted',
  'shortName': 'Stansted',
  'municipalityName': 'London',
  'location': {'lat': 51.885, 'lon': 0.234999},
  'countryCode': 'GB'},
 {'icao': 'EGLL',
  'iata': 'LHR',
  'name': 'London, London Heathrow',
  'shortName': 'Heathrow',
  'municipalityName': 'London',
  'location': {'lat': 51.4706, 'lon': -0.461941},
  'countryCode': 'GB'},
 {'icao': 'EGLC',
  'iata': 'LCY',
  'name': 'London, London City',
  'shortName': 'City',
  'municipalityName': 'London',
  'location': {'lat': 51.5053, 'lon': 0.055277},
  'countryCode': 'GB'},
 {'icao': 'EGKK',
  'iata': 'LGW',
  'name': 'London, London Gatwick',
  'shortName': 'Gatwick',
  'municipalityName': 'London',
  'location': {'lat': 51.1481, 'lon': -0.190277},
  'countryCode': 'GB'},
 {'icao': 'EGGW',
  'iata': 'LTN',
  'name': 'London, London Luton',
  'shortName': 'Luton',
  'municipalityName': 'London',
  'location': {'lat': 51.8747, 'lon': -0.368333},
  'countryCode': '